In [1]:
import sys
import numpy as np
import pandas as pd
import statsmodels.api as sm
import os
import warnings
import folium
import branca

warnings.filterwarnings("ignore")

In [3]:
df_filled_23 = pd.read_pickle(r'D:\capstone_F24\data\Cleaned_data\df_filled_23.pkl')

In [4]:
df_filled_23

timestamp,2023-02-01 00:00:00,2023-02-01 01:00:00,2023-02-01 02:00:00,2023-02-01 03:00:00,2023-02-01 04:00:00,2023-02-01 05:00:00,2023-02-01 06:00:00,2023-02-01 07:00:00,2023-02-01 08:00:00,2023-02-01 09:00:00,...,2023-12-31 14:00:00,2023-12-31 15:00:00,2023-12-31 16:00:00,2023-12-31 17:00:00,2023-12-31 18:00:00,2023-12-31 19:00:00,2023-12-31 20:00:00,2023-12-31 21:00:00,2023-12-31 22:00:00,2023-12-31 23:00:00
station_complex_id,,,,,,,,,,,,,,,,,,,,,
1,27.0,16.0,15.0,16.0,47.0,227.0,654.0,1528.0,2208.0,1020.0,...,383.0,334.0,334.0,357.0,406.0,307.0,335.0,319.0,234.0,63.0
10,167.0,46.0,26.0,10.0,8.0,25.0,84.0,222.0,490.0,399.0,...,1.0,4.0,3.0,1.0,4.0,2.0,5.0,3.0,5.0,2.0
100,13.0,2.0,1.0,2.0,6.0,15.0,75.0,141.0,281.0,161.0,...,81.0,66.0,60.0,50.0,71.0,74.0,101.0,84.0,54.0,26.0
101,59.0,30.0,13.0,6.0,18.0,61.0,230.0,635.0,1223.0,765.0,...,353.0,364.0,376.0,364.0,290.0,318.0,302.0,270.0,217.0,107.0
118,94.0,25.0,15.0,3.0,4.0,9.0,49.0,95.0,188.0,114.0,...,212.0,231.0,249.0,236.0,292.0,286.0,309.0,320.0,326.0,148.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,15.0,3.0,6.0,13.0,39.0,87.0,233.0,566.0,768.0,433.0,...,193.0,197.0,147.0,143.0,182.0,191.0,164.0,141.0,114.0,55.0
96,13.0,6.0,5.0,8.0,24.0,72.0,193.0,421.0,621.0,364.0,...,175.0,157.0,121.0,136.0,153.0,214.0,159.0,139.0,105.0,51.0
97,31.0,14.0,19.0,22.0,37.0,118.0,290.0,871.0,1367.0,837.0,...,337.0,303.0,282.0,301.0,289.0,341.0,265.0,338.0,207.0,122.0


In [5]:
time_cols = df_filled_23.columns

In [6]:
df_23_pred = pd.read_pickle('df_23_pred_LSTM.pkl')

In [7]:
df_23_pred

,station_complex_id,predicted_ridership_2023,predicted_timestamps
0,1,"[[51.460418701171875], [37.575199127197266], [...","[2023-02-02 00:00:00, 2023-02-02 01:00:00, 202..."
1,10,"[[85.66735076904297], [35.20859146118164], [-9...","[2023-02-02 00:00:00, 2023-02-02 01:00:00, 202..."
2,100,"[[7.264353275299072], [8.049354553222656], [-0...","[2023-02-02 00:00:00, 2023-02-02 01:00:00, 202..."
3,101,"[[41.19313430786133], [30.327926635742188], [1...","[2023-02-02 00:00:00, 2023-02-02 01:00:00, 202..."
4,118,"[[102.69548797607422], [29.632213592529297], [...","[2023-02-02 00:00:00, 2023-02-02 01:00:00, 202..."
...,...,...,...
356,95,"[[-12.225086212158203], [-2.7507498264312744],...","[2023-02-02 00:00:00, 2023-02-02 01:00:00, 202..."
357,96,"[[2.718946695327759], [9.364445686340332], [7....","[2023-02-02 00:00:00, 2023-02-02 01:00:00, 202..."
358,97,"[[0.9972857236862183], [-4.06417179107666], [-...","[2023-02-02 00:00:00, 2023-02-02 01:00:00, 202..."
359,98,"[[23.597551345825195], [24.310914993286133], [...","[2023-02-02 00:00:00, 2023-02-02 01:00:00, 202..."


In [8]:
df_23_pred = df_23_pred.drop(columns='predicted_timestamps')

In [9]:
df_23_pred.columns

Index(['station_complex_id', 'predicted_ridership_2023'], dtype='object')

In [10]:
list_size = df_23_pred[df_23_pred['station_complex_id']=='1']['predicted_ridership_2023'].values.tolist()

In [11]:
len(list_size[0])

7992

In [12]:
# Set index to station_complex_id
df_23_pred = df_23_pred.set_index("station_complex_id")

# Expand the lists in predicted_ridership_2023 into new columns
expanded_23_pred = pd.DataFrame(df_23_pred["predicted_ridership_2023"].tolist(), 
                           index=df_23_pred.index, 
                           columns=time_cols)

expanded_23_pred

ValueError: 8016 columns passed, passed data had 7992 columns

In [ ]:
expanded_23_pred.to_pickle('expanded_23_pred_LSTM.pkl')

In [ ]:
# Reset index to include station_complex_id as a column
preds_long = expanded_23_pred.reset_index().melt(
    id_vars=["station_complex_id"],  # Columns to keep as identifiers
    var_name="timestamp",           # Name for the 'wide' columns (timestamps)
    value_name="ridership"          # Name for the values in those columns
)

# Convert the timestamp column to a proper datetime type if necessary
preds_long["timestamp"] = pd.to_datetime(preds_long["timestamp"])

preds_long

In [ ]:
# Reset index to include station_complex_id as a column
actual_long = df_filled_23.reset_index().melt(
    id_vars=["station_complex_id"],  # Columns to keep as identifiers
    var_name="timestamp",           # Name for the 'wide' columns (timestamps)
    value_name="ridership"          # Name for the values in those columns
)

# Convert the timestamp column to a proper datetime type if necessary
actual_long["timestamp"] = pd.to_datetime(actual_long["timestamp"])

actual_long

In [ ]:
# Rename the 'ridership' column in preds_long to 'predicted'
preds_long = preds_long.rename(columns={"ridership": "predicted"})

# Rename the 'ridership' column in actual_long to 'actual'
actual_long = actual_long.rename(columns={"ridership": "actual"})

# Merge preds_long with actual_long on 'station_complex_id' and 'timestamp'
pred_vs_actual = actual_long.merge(
    preds_long[["station_complex_id", "timestamp", "predicted"]],  # Select relevant columns from preds_long
    on=["station_complex_id", "timestamp"],                       # Merge on these columns
    how="inner"                                                   # Use inner join to keep only matching rows
)

pred_vs_actual

In [ ]:
df_filled_22 = pd.read_pickle(r'D:\capstone_F24\data\Cleaned_data\df_filled_22.pkl')

In [ ]:
df_filled_22

In [ ]:
# Reset index to include station_complex_id as a column
actual_22_long = df_filled_22.reset_index().melt(
    id_vars=["station_complex_id"],  # Columns to keep as identifiers
    var_name="timestamp",           # Name for the 'wide' columns (timestamps)
    value_name="ridership"          # Name for the values in those columns
)

# Convert the timestamp column to a proper datetime type if necessary
actual_22_long["timestamp"] = pd.to_datetime(actual_22_long["timestamp"])

actual_22_long

In [ ]:
# Rename the 'ridership' column in actual_long to 'actual'
pred_vs_actual = pred_vs_actual.rename(columns={"actual": "actual_23", "predicted":"pred_23"})

pred_vs_actual

In [ ]:
df_station_info = pd.read_csv(r"D:\capstone_F24\data\MTA_Subway_Stations_20241002.csv")

In [ ]:
df_station_info

In [ ]:
df_station_info['Daytime Routes'].unique()

In [ ]:
# Cast Complex ID to string in df_station_info
df_station_info['Complex ID'] = df_station_info['Complex ID'].astype(str)

# Remove duplicates in df_station_info based on Complex ID
df_station_info_unique = df_station_info.drop_duplicates(subset=['Complex ID'])

# Select relevant columns from df_station_info
station_info_cols = ['Complex ID', 'Stop Name', 'Borough', 'Daytime Routes', 
                     'Structure', 'GTFS Latitude', 'GTFS Longitude']
df_station_info_selected = df_station_info_unique[station_info_cols]

# Perform the merge
merged_df = pred_vs_actual.merge(
    df_station_info_selected, 
    left_on='station_complex_id', 
    right_on='Complex ID', 
    how='left'
)

# Drop duplicate 'Complex ID' column if desired
merged_df = merged_df.drop(columns=['Complex ID'])

merged_df

In [ ]:
# Rename 'GTFS Latitude' and 'GTFS Longitude' columns to 'lat' and 'long' respectively
merged_df.rename(columns={'GTFS Latitude': 'lat', 'GTFS Longitude': 'long', 'Stop Name': 'stop_name', 'Borough': 'borough','Structure': 'structure'}, inplace=True)

# Create proxy variable 'is_weekend' which determines if the timestamp is a weekend (1 for weekend, 0 for weekday)
merged_df['timestamp'] = pd.to_datetime(merged_df['timestamp'])  # Ensure timestamp is in datetime format
merged_df['is_weekend'] = merged_df['timestamp'].dt.weekday >= 5  # 5 and 6 represent Saturday and Sunday
merged_df['is_weekend'] = merged_df['is_weekend'].astype(int)  # Convert boolean to integer (1 for weekend, 0 for weekday)

# Create proxy variable 'day_of_week' which gets the day of the week as an integer (0=Monday, 6=Sunday)
merged_df['day_of_week'] = merged_df['timestamp'].dt.weekday

# Refactor 'Daytime Routes' column into a new column 'routes' which is a list of elements split by space
merged_df['routes'] = merged_df['Daytime Routes'].str.split()

# Add new column 'num_lines_serviced' which is the length of each list in the 'routes' column
merged_df['num_lines_serviced'] = merged_df['routes'].apply(len)

# Drop the 'Daytime Routes' column
merged_df.drop(columns=['Daytime Routes'], inplace=True)


In [ ]:
merged_df

In [ ]:
# Assuming your DataFrame is named df
merged_df['diff_23'] = merged_df['pred_23'] - merged_df['actual_23']

In [ ]:
merged_df

In [ ]:
import sys
!{sys.executable} -m pip install ipywidgets

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import ipywidgets as widgets
from ipywidgets import interactive

In [ ]:
def plot_station(station_complex_id):
    station_data = merged_df[merged_df['station_complex_id'] == station_complex_id]

    # Time series plot for predicted vs actual
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=station_data['timestamp'], y=station_data['pred_23'], mode='lines', name='Predicted Ridership'))
    fig.add_trace(go.Scatter(x=station_data['timestamp'], y=station_data['actual_23'], mode='lines', name='Actual Ridership'))
    fig.update_layout(title=f"Predicted vs Actual Ridership - {station_complex_id}", xaxis_title="Date", yaxis_title="Ridership")
    
    fig_diff = go.Figure()
    fig_diff.add_trace(go.Scatter(x=station_data['timestamp'], y=station_data['diff_23'], mode='lines', name='Difference (Pred - Actual)', line=dict(color='green' if station_data['diff_23'].iloc[0] > 0 else 'red')))
    fig_diff.update_layout(title=f"Difference in Ridership - {station_complex_id}", xaxis_title="Date", yaxis_title="Difference")
    
    fig.show()
    fig_diff.show()

# Dropdown for station selection
station_dropdown = widgets.Dropdown(
    options=merged_df['station_complex_id'].unique(),
    description='Station:',
)

interactive(plot_station, station_complex_id=station_dropdown)